# INM702 Coursework Task 2
### Aaron Mir (Student Number: 160001207)

<img src="Task2Overview.jpg" alt="Task2Overview" width="700"/>

The second task is about classifying handwritten digits. We will use the MNIST dataset for training and testing. The point of this task is to develop a multi-layer neural network for classification using mostly Numpy:


    • Implement sigmoid and relu layers (with forward and backward pass)

    • Implement a softmax output layer
    
    • Implement a fully parameterizable neural network (number and types of layers, number of units)

    • Implement an optimizer(e.g. SGD or Adam)and a stopping criterion of your choosing

    • Train your Neural Network using backpropagation

Evaluate different neural network architectures and compare your different results. e.g. fully relu vs fully sigmoid, number of layers etc.

You can also compare withthe results presented in http://yann.lecun.com/exdb/mnist/